In [ ]:
## Sealnet -- deployment

---

This jupyter allows you to make predictions on your own images with the seal detection pipeline. *WARNING* : CNNs used here were trained on panchromatic WorldView-3 imagery so results may not be satisfactory on different imagery / color bands. 

### Load packages

In [1]:
import os

### Call detection pipeline 

Though you are free to try different combinations, the current setup of CNNs provides the best results on our test images. 

In [ ]:
# select training set, model and hyperparameters
mod_arches = ['UnetCntWRNOccDense']
t_dir = 'training_set_vanilla'
hyp_set = 'F'

# setup input folder ** change here **
inp_fldr = '/home/bento/testing_scenes'

# loop through architectures
for mod_arch in mod_arches:
    out_fldr = 'test_predictions_{}'.format(mod_arch)

    # loop through images
    inp_imgs = [img for img in os.listdir('{}'.format(inp_fldr)) if img[-4:] == '.tif']
    for img in inp_imgs:
        img_path = '{}/{}'.format(inp_fldr, img)
        out_fldr_img = '{}/{}'.format(out_fldr, img)
        # call pipeline 
        !python predict_raster.py --input_image=$img_path \
                                  --model_architecture=$mod_arch \
                                  --hyperparameter_set=$hyp_set \
                                  --training_set=$t_dir --test_folder=$out_fldr_img \
                                  --stride=0.5


127949 tiles created in 1 minutes and 17.92 seconds
Testing complete in 0h 49m 34s
Total predicted in WV03_20170204000736_104001002678E600_17FEB04000736-P1BS-501513717060_01_P001_u08rf3031.tif:  72

138298 tiles created in 1 minutes and 26.89 seconds


### Testing predictions

In [13]:
# setup output folder ** change here **
mod_arches = ['Unet', 'UnetCntWRN', 'UnetCntWRNOccDense']
for mod_arch in mod_arches:
    inp_fldr = 'test_predictions_{}'.format(mod_arch)
    out_fldr = '{}/merged_shapefiles'.format(inp_fldr)

    # merge shapefiles
    !python merge_shapefiles.py --input_dir=$inp_fldr --output_dir=$out_fldr

    # test predictions
    shp_prediction = '{}/merged_locations.dbf'.format(out_fldr)
    shp_ground_truth = 'shape_files/seal-points-consensus.dbf'
    output_file = '{}/{}_'.format(inp_fldr, mod_arch)

    !python test_sealnet.py --shp_prediction=$shp_prediction \
                            --shp_ground_truth=$shp_ground_truth \
                            --tolerance_seal=5 \
                            --tolerance_haul=20 \
                            --output_file=$output_file